In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
#warnings.filterwarnings('ignore')


def clean_real_estate_data(df):
    print(" Starting data cleaning...")
    print(f"Original data shape: {df.shape}")

    # Step 1: Remove completely empty rows and columns
    print("\n Step 1: Removing empty rows and columns...")

    # Remove  Url row
    df = df.drop("url", axis="columns")
    # Remove rows where all values are NaN
    df = df.dropna(how="all")

    # Remove columns where all values are NaN
    df = df.dropna(axis=1, how="all")

    print(f"After removing empty: {df.shape}")

    # Step 2: Handle the unnamed index column
    print("\n Step 2: Cleaning column names...")

    # Drop the unnamed index column if it exists
    if "Unnamed: 0" in df.columns:
        df = df.drop("Unnamed: 0", axis=1)
        print("Removed 'Unnamed: 0' column")

    # Remove specified columns based on decisions
    print("\n Step 2.1: Removing specified columns...")
    columns_to_remove = [
        'heatingType',
        'hasThermicPanels', 
        'gardenOrientation', 
        'terraceOrientation',
        'roomCount', 
        'monthlyCost', 
        'diningRoomSurface', 
        'streetFacadeWidth', 
        'kitchenSurface', 
        'hasBalcony', 
        'accessibleDisabledPeople'
    ]
    
    columns_removed = []
    for col in columns_to_remove:
        if col in df.columns:
            df = df.drop(col, axis=1)
            columns_removed.append(col)
    
    if columns_removed:
        print(f"Removed specified columns: {columns_removed}")
    else:
        print("None of the specified columns were found in the dataset")

    # NEW: Merge parking columns
    print("\n Step 2.2: Merging parking columns...")
    if 'parkingCountOutdoor' in df.columns and 'parkingCountIndoor' in df.columns:
        # Fill NaN values with 0 before merging
        df['parkingCountOutdoor'] = df['parkingCountOutdoor'].fillna(0)
        df['parkingCountIndoor'] = df['parkingCountIndoor'].fillna(0)
        
        # Create merged parking count
        df['totalParkingCount'] = df['parkingCountOutdoor'] + df['parkingCountIndoor']
        
        # Drop original parking columns
        df = df.drop(['parkingCountOutdoor', 'parkingCountIndoor'], axis=1)
        print(f"Merged parking columns into 'totalParkingCount': {df['totalParkingCount'].value_counts().to_dict()}")
    else:
        print("Parking columns not found for merging")

    # Step 3: Remove duplicates based on ID
    print("\n Step 3: Removing duplicate properties...")

    if "id" in df.columns:
        before_dup = len(df)
        df = df.drop_duplicates(subset=["id"], keep="first")
        after_dup = len(df)
        print(f"Removed {before_dup - after_dup} duplicate properties")

    # Step 4: Clean price column (main target variable)
    print("\n Step 4: Cleaning price data...")

    if "price" in df.columns:
        # Remove rows with missing prices (can't train without target)
        before_price = len(df)
        df = df.dropna(subset=["price"])
        after_price = len(df)
        print(f"Removed {before_price - after_price} rows with missing prices")

        # Remove unrealistic prices (too low or too high)
        # Assuming Belgian real estate: min 50k, max 5M euros
        #df = df[(df["price"] >= 50000) & (df["price"] <= 5000000)]
        #print(f"Kept prices between 50k and 5M euros: {len(df)} properties")

    # NEW: Handle garden surface imputation
    print("\n Step 4.1: Handling garden surface imputation...")
    if 'gardenSurface' in df.columns and 'hasGarden' in df.columns:
        # Convert hasGarden to boolean if it's not already
        df['hasGarden'] = df['hasGarden'].map({True: True, False: False, 'True': True, 'False': False})
        
        # Impute gardenSurface with 0 where hasGarden is False
        mask = df['hasGarden'] == False
        before_impute = df.loc[mask, 'gardenSurface'].isna().sum()
        df.loc[mask, 'gardenSurface'] = df.loc[mask, 'gardenSurface'].fillna(0)
        after_impute = df.loc[mask, 'gardenSurface'].isna().sum()
        
        print(f"Imputed {before_impute - after_impute} gardenSurface values with 0 where hasGarden=False")
        print(f"Garden surface stats: min={df['gardenSurface'].min()}, max={df['gardenSurface'].max()}, median={df['gardenSurface'].median()}")
    else:
        print("gardenSurface or hasGarden columns not found")

    # Step 5: Clean numeric columns
    print("\n Step 5: Cleaning numeric features...")

    numeric_cols = ["bedroomCount", "bathroomCount", "toiletCount", "terraceSurface", "totalParkingCount"]

    for col in numeric_cols:
        if col in df.columns:
            # Replace negative values with NaN
            df.loc[df[col] < 0, col] = np.nan

            # Replace unrealistic high values
            if col in ["bedroomCount", "bathroomCount"]:
                df.loc[df[col] > 10, col] = np.nan  # Max 10 bedrooms/bathrooms
            elif col == "toiletCount":
                df.loc[df[col] > 5, col] = np.nan  # Max 5 toilets                               ############################################
            elif col == "totalParkingCount":
                df.loc[df[col] > 10, col] = np.nan  # Max 10 parking spaces

            print(f"Cleaned {col}: {df[col].notna().sum()} valid values")
    # facedecount can be 4 max
    if "facedeCount" in df.columns:
        facedes={1.0:1,2.0:2,3.0:3,4.0:4,5.0:4,6.0:4,7.0:4,8.0:4,9.0:4,10.0:4,16.0:4,86.0:4}
        df["facedeCount"]=df["facedeCount"].map(facedes)

    # Step 6: Clean categorical columns
    print("\n Step 6: Cleaning categorical features...")

    # Clean property type and subtype - REMOVE ALL SPACES
    if "type" in df.columns:
        df["type"] = df["type"].str.upper().str.strip().str.replace(" ", "")
        print(f"Property types: {df['type'].value_counts().to_dict()}")

    if "subtype" in df.columns:
        df["subtype"] = df["subtype"].str.upper().str.strip().str.replace(" ", "")

    # Clean location data - REMOVE ALL SPACES
    location_cols = ["province", "locality"]
    for col in location_cols:
        if col in df.columns:
            df[col] = df[col].str.strip().str.title().str.replace(" ", "")
            print(f"Unique {col}s: {df[col].nunique()}")

    # Clean ALL text columns - remove spaces everywhere
    text_cols = df.select_dtypes(include=["object"]).columns
    for col in text_cols:
        if col not in ["url"]:  # Keep URLs as they are
            df[col] = df[col].astype(str).str.strip().str.replace(" ", "")
            print(f"Removed spaces from {col}")
            
        
    # Step 7: Handle boolean columns (updated list without removed columns)
    print("\n Step 7: Cleaning boolean features...")

    boolean_cols = [
        "hasAttic",
        "hasBasement",
        "hasDressingRoom",
        "hasDiningRoom",
        "hasLift",
        "hasHeatPump",
        "hasPhotovoltaicPanels",
        "hasTerrace",
        "hasAirConditioning",
        "hasArmoredDoor",
        "hasVisiophone",
        "hasOffice",
        "hasSwimmingPool",
        "hasFireplace",
        "hasLivingRoom",
        "hasGarden" 
    ]

    for col in boolean_cols:
        if col in df.columns:
            # Convert to proper boolean (True/False/NaN)
            df[col] = df[col].map(
                {True: True, False: False, "True": True, "False": False}
            )
            #df[col] = df[col].fillna(False)  # Convert any remaining NaN to False
            #print(f"{col}: {df[col].value_counts(dropna=False).to_dict()}")

    # Step 8: Clean energy performance (EPC)
    print("\n Step 8: Cleaning energy performance...")
    if "epcScore" in df.columns:
        epc_scores={"A++":"A","A+":"A","A":"A","B":"B","C":"C","D":"D","E":"E","F":"F","G":"G","G_C":"G","F_D":"F","E_C":"E","C_B":"C","E_D":"E","F_C":"F","C_A":"C","G_F":"G","G_E":"G","D_C":"D"}
        df["epcScore"]=df["epcScore"].map(epc_scores)
    #if "epcScore" in df.columns:
        '''valid_epc = ["A", "B", "C", "D", "E", "F", "G"]
        df.loc[~df["epcScore"].isin(valid_epc), "epcScore"] = np.nan
        print(f"EPC distribution: {df['epcScore'].value_counts().to_dict()}")'''
        
    # Step 8.1 : fill nulls in bedroom count with value in room count
     
   # if 'bedroomCount' in df.columns and 'roomCount' in df.columns:
        # fill bedroom count to 1 if room count is not null
      #  df.loc[df['roomCount'].notna() & df['bedroomCount'].isna(), 'bedroomCount'] = 1

    # Step 9: Remove columns with too many missing values
    print("\n Step 9: Removing columns with too many missing values...")

    # Remove columns where more than 80% of values are missing
    threshold = 0.5
    missing_pct = df.isnull().sum() / len(df)
    cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()

    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
        print(f"Removed columns with >80% missing: {cols_to_drop}")

    # Step 10: Final summary
    print("\n Data cleaning completed!")
    print(f"Final data shape: {df.shape}")
    print(f"Columns kept: {list(df.columns)}")

    # Show missing values summary
    print("\n Missing values summary:")
    missing_summary = df.isnull().sum()
    missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
    for col, missing_count in missing_summary.items():
        pct = (missing_count / len(df)) * 100
        print(f"  {col}: {missing_count} ({pct:.1f}%)")

    return df


In [2]:
if __name__ == "__main__":

    # 1. Load your data:
    df = pd.read_csv("data/final_dataset.csv")

    # 2. Clean the data:
    df_clean = clean_real_estate_data(df)

    # 3. Apply categorical encoding:
   # df_encoded, encoding_mappings = full_encoding_pipeline(df_clean)

    # 4. Save the results:
    df_clean.to_csv("data/cleaned_data1.csv", index=False)
    #df_encoded.to_csv("ml_ready_real_estate_data.csv", index=False)

    #print(f"\n=== FINAL RESULTS ===")
    #print(f"Cleaned data saved: cleaned_data1.csv ({df_clean.shape})")
    #print(f"ML-ready data saved: ml_ready_real_estate_data.csv ({df_encoded.shape})")
    #print(f"Encoding mappings: {list(encoding_mappings.keys())}")

 Starting data cleaning...
Original data shape: (80368, 53)

 Step 1: Removing empty rows and columns...
After removing empty: (80368, 49)

 Step 2: Cleaning column names...
Removed 'Unnamed: 0' column

 Step 2.1: Removing specified columns...
Removed specified columns: ['heatingType', 'hasThermicPanels', 'gardenOrientation', 'terraceOrientation', 'roomCount', 'diningRoomSurface', 'streetFacadeWidth', 'kitchenSurface']

 Step 2.2: Merging parking columns...
Merged parking columns into 'totalParkingCount': {0.0: 44853, 1.0: 17610, 2.0: 7457, 3.0: 3308, 4.0: 2356, 5.0: 1402, 6.0: 1060, 7.0: 568, 8.0: 369, 10.0: 238, 9.0: 167, 11.0: 145, 12.0: 126, 13.0: 70, 14.0: 54, 15.0: 51, 20.0: 48, 16.0: 43, 17.0: 34, 30.0: 28, 18.0: 28, 22.0: 24, 50.0: 17, 23.0: 16, 79.0: 14, 21.0: 13, 25.0: 13, 24.0: 13, 19.0: 13, 36.0: 13, 32.0: 12, 40.0: 11, 28.0: 10, 27.0: 10, 100.0: 9, 26.0: 9, 29.0: 9, 117.0: 8, 34.0: 8, 67.0: 5, 35.0: 5, 60.0: 4, 109.0: 4, 31.0: 4, 87.0: 4, 53.0: 4, 44.0: 4, 42.0: 3, 120.0: 